In [10]:
# import packages
from owslib.wcs import WebCoverageService
import geopandas as gpd

In [11]:
# link to soil organic carbon stock from SoilGrids
wcs = WebCoverageService('https://maps.isric.org/mapserv?map=/map/ocs.map',
                         version='2.0.1')

# list available layers
list(wcs.contents)

['ocs_0-30cm_Q0.05',
 'ocs_0-30cm_Q0.5',
 'ocs_0-30cm_Q0.95',
 'ocs_0-30cm_mean',
 'ocs_0-30cm_uncertainty']

In [12]:
# list operations available
[op.name for op in wcs.operations]

['GetCapabilities', 'DescribeCoverage', 'GetCoverage']

In [13]:
# check coverage for one of the layers
ocs = wcs.contents['ocs_0-30cm_Q0.05']

# # supported CRS
# print(ocs.supportedCRS)

# supported format
print(ocs.supportedFormats)

# bounding box
print(ocs.boundingboxes)

['image/tiff', 'image/png', 'image/jpeg', 'image/png; mode=8bit', 'image/vnd.jpeg-png', 'image/vnd.jpeg-png8']
[{'nativeSrs': 'http://www.opengis.net/def/crs/EPSG/0/152160', 'bbox': (-19949750.0, -6147500.0, 19861750.0, 8361000.0)}]


In [14]:
# import y2y shapefile
y2y = gpd.read_file('./study_area/Y2Y_RegionBoundary_Final2013/Y2Y_RegionBoundary.shp')

# # list standard crs we want to us (NAD83)
# target_crs = 'EPSG:4269'

# convert y2y crs
y2y = y2y.to_crs("+proj=igh +datum=WGS84 +no_defs +towgs84=0,0,0")
print(y2y.crs)

# get bounding box of whole region
bbox = y2y.total_bounds
bbox


+proj=igh +datum=WGS84 +no_defs +towgs84=0,0,0 +type=crs


array([-13538108.38001439,   4647455.57561753, -11820893.76105979,
         7209306.68616891])

In [27]:
# format bounding box into subset for WCS 2.0
# something didn't work the first time so going to redownload all of them with a small buffer
# just to get a slightly different output
subsets = [('X', float(bbox[0]-100), float(bbox[2]+100)), ('Y', float(bbox[1]), float(bbox[3]))]
subsets

[('X', -13538208.38001439, -11820793.761059787),
 ('Y', 4647455.575617532, 7209306.68616891)]

In [28]:
# loop through layers and download each
for layer in wcs.contents:

    # fetch map segment for layer
    response = wcs.getCoverage(identifier=[layer],
                            crs='http://www.opengis.net/def/crs/EPSG/0/152160',
                            subsets=subsets,
                            resx=250,
                            resy=250,
                            format='image/tiff')
    
    # download data
    with open('./carbon_stock_data/soc_soilgrids/' + layer + '.tif', 'wb') as file:
        file.write(response.read())